# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價(MedV)。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [3]:
boston = dataset("MASS", "Boston")
# first(boston, 10)
describe(boston)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Crim,3.61352,0.00632,0.25651,88.9762,,,Float64
2,Zn,11.3636,0.0,0.0,100.0,,,Float64
3,Indus,11.1368,0.46,9.69,27.74,,,Float64
4,Chas,0.06917,0,0.0,1,,,Int64
5,NOx,0.554695,0.385,0.538,0.871,,,Float64
6,Rm,6.28463,3.561,6.2085,8.78,,,Float64
7,Age,68.5749,2.9,77.5,100.0,,,Float64
8,Dis,3.79504,1.1296,3.20745,12.1265,,,Float64
9,Rad,9.54941,1,5.0,24,,,Int64


In [7]:
show(boston,allcols=true)

506×14 DataFrame
│ Row │ Crim    │ Zn      │ Indus   │ Chas  │ NOx     │ Rm      │ Age     │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ 0.00632 │ 18.0    │ 2.31    │ 0     │ 0.538   │ 6.575   │ 65.2    │
│ 2   │ 0.02731 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 6.421   │ 78.9    │
│ 3   │ 0.02729 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 7.185   │ 61.1    │
│ 4   │ 0.03237 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.998   │ 45.8    │
│ 5   │ 0.06905 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 7.147   │ 54.2    │
│ 6   │ 0.02985 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.43    │ 58.7    │
│ 7   │ 0.08829 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.012   │ 66.6    │
│ 8   │ 0.14455 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.172   │ 96.1    │
│ 9   │ 0.21124 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 5.631   │ 100.0   │
│ 10  │ 0.17004 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.004   │ 85.9 

In [5]:
ind = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(ind,at=0.7)
train = boston[train_ind,:]
test = boston[test_ind,:];

In [27]:
fm2 = @formula(MedV ~ Tax + Crim + Black + PTRatio + Zn + Dis + NOx +Indus +Chas+Age)
logreg = glm(fm2, train, Normal(), IdentityLink()) # returns coefficients
pred = predict(logreg, test)

152-element Array{Union{Missing, Float64},1}:
 33.99917306066006
 13.339749995086464
 22.901122330327517
 28.72156382706399
 18.00379233656261
 31.997364164199112
 15.687635509132505
 28.151403795989552
  9.653016640748048
 14.917773299613081
 36.56398630444713
 21.217511827780555
 29.3319935654194
  ⋮
 16.809034712530146
 25.388203091845533
 29.571551194611487
 37.06567186680171
 21.805228854765943
 19.546379208000232
 27.955636711647145
 28.846777137212108
 14.705644236903055
 27.607627670917363
 28.343499451133507
 15.204353973950337

In [28]:
using Statistics
rmse(x,y) = sqrt(mean((x - y).^2))
# error(x, y) = ((x > 0.5) ? 1.0 : 0.0) == y
# accuracy(xs, ys) = mean(error.(xs, ys))
rmse(pred, test[!, :MedV])

6.2664920228938765